# Dataset Generation Notebook

## Description

This notebook serves as a collection of functions which were used during the project, to create point clouds from meshes. Approaches presented in this notebook were used to generate trainings data for our custom neural network.

**Important note:** open3d has **a lot** of issues with specific python verison. Please use the **python 3.10.15** version together with the open3d 0.18 version from our conda environment if you run into any issues.

As an example we will use a mesh from the THuman2.1 dataset.


# Table of Contents
- [0. Imports](#0)
- [1.0 Mesh to Point Cloud](#1.0)
  - [1.1 Functions](#1.1)
  - [1.2 Example usage](#1.2)
- [2.0 Generate POV Point Clouds](#2.0)
  - [2.1 Functions](#2.1)
  - [2.2 Example usage](#2.2)
- [3.0 Point Cloud to Voxel Grid](#3.0)
  - [3.1 Functions](#3.1)
  - [3.2 Example usage](#3.2)
- [4.0 Voxel to Truncated Disatnce Field](#4.0)
  - [4.1 Functions](#4.1)
  - [4.2 Example usage](#4.2)


## 0. Imports <a name="0"></a>

Relevant imports use in the data preperation

In [1]:
import os
import copy
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## 1.0 Mesh to Point Cloud <a name="1.0"></a>

In this section we demonatrate how to load a mesh file, how to transform it into a point cloud and how to visualize and safe the resulting point cloud.

### 1.1 Functions <a name="1.1"></a>

In [2]:
# ==================================================================================================
#  Load Mesh
# ==================================================================================================
def load_mesh(file_path):
    """
    desctiption:
        loads mesh file (ie. ".obj" file)
    input: 
        file_path: file path to mesh file
    output:
        loaded open3d mesh object
    """
    if not os.path.exists(file_path):
        print(f"Error: The file '{file_path}' does not exist.")
        return None
    
    mesh = o3d.io.read_triangle_mesh(file_path)
    if mesh.is_empty():
        print("Empty mesh file detected. Please double check path or file contents.")
        return None
    
    return mesh


# ==================================================================================================
#  Draw Mesh
# ==================================================================================================
def draw_mesh(mesh):
    """
    description:
        visualizes an open3d mesh object
        before visualization the normals of the object will be calculated
    input:
        mesh: open3d mesh object
    """
    if mesh is None:
        print("Error: Mesh object is None.")
        return
    
    if mesh.is_empty():
        print("Error: Empty mesh object detected.")
    
    mesh.compute_vertex_normals()
    o3d.visualization.draw_geometries([mesh], window_name="Example Mesh", mesh_show_back_face=True)


# ==================================================================================================
#  Mesh to Point Cloud
# ==================================================================================================
def mesh_to_pcd(mesh, num_points=10000):
    """
    description:
        creates a point cloud from mesh file with poisson disk method
    input:
        mesh:       open3d mesh object
        num_points: number of points which the resulting point cloud should have
    output:
        open3d point cloud object
    """
    if mesh is None:
        print("Error: Mesh object is None.")
        return None
    
    if mesh.is_empty():
        print("Error: Empty mesh object detected.")
        return None
    
    pcd = mesh.sample_points_poisson_disk(number_of_points=num_points)
    if pcd.is_empty():
        print("Error: Empty point cloud object detected.")
        return None
    
    return pcd


# ==================================================================================================
#  Draw Point Cloud
# ==================================================================================================
def draw_pcd(pcd):
    """
    description:
        visualizes an open3d point cloud object
    input:
        pcd: open3d point cloud object
    """
    if pcd is None:
        print("Error: Point cloud object is None.")
        return
    
    if pcd.is_empty():
        print("Error: Empty point cloud object detected.")
        return
    
    o3d.visualization.draw_geometries([pcd], window_name="Example PCD", point_show_normal=False)


# ==================================================================================================
#  Save Point Cloud
# ==================================================================================================
def save_pcd(pcd, file_path):
    """
    description:
        saves open3d point cloud to given file path
    input:
        pcd:        open3d point cloud object to be saved
        file_path:  file path where the point cloud will be stored (".ply" file format)
    """
    if pcd is None:
        print("Error: Point cloud object is None.")
        return
    
    if pcd.is_empty():
        print("Error: Empty point cloud object detected.")
        return

    o3d.io.write_point_cloud(file_path, pcd)

### 1.2 Example usage <a name="1.1"></a>

In [3]:
# File paths
example_mesh_file = "example_data/0000.obj"
example_pcd_file = "example_data/0000.ply"

# Example usage
example_mesh = load_mesh(example_mesh_file)
draw_mesh(example_mesh)
example_pcd = mesh_to_pcd(example_mesh)
draw_pcd(example_pcd)
save_pcd(example_pcd, example_pcd_file)

## 2.0 Generate POV Point Clouds <a name="2.0"></a>

In this section we show how to generate point clouds which only consist of visual points from a camera perspective. All points which hide behind a surface and thus are not visible from the camera angle will be removed.

### 2.1 Functions <a name="2.1"></a>

In [4]:
# ==================================================================================================
#  Load Point Cloud
# ==================================================================================================
def load_pcd(file_path):
    """
    description:
        loads point cloud file (i.e. ".ply" file)
    input:
        file_path: file path to point cloud file
    output:
        loaded open3d point cloud object
    """
    if not os.path.exists(file_path):
        print(f"Error: The file '{file_path}' does not exist.")
        return None

    pcd = o3d.io.read_point_cloud(file_path)
    if pcd.is_empty():
        print("Empty pcd file detected. Please double check path or file contents.")
    
    return pcd


# ==================================================================================================
#  Rotate Point Cloud
# ==================================================================================================
def rotate_pcd(pcd, rotation_angle_degree=0):
    """
    description:
        generates a rotated version of given point cloud
        rotation happens along the y-axis
        the returned point cloud is a copy, the input will not be changed
    input:
        pcd:                    open3d point cloud to be rotated
        rotation_angle_degree:  rotation angle in degree
    output:
        rotated copy of the given point cloud
    """
    if pcd is None:
        print("Error: Point cloud object is None.")
        return None
    
    if pcd.is_empty():
        print("Error: Empty point cloud object detected.")
        return None
    
    rotation_angle_radians = np.deg2rad(rotation_angle_degree)
    R = pcd.get_rotation_matrix_from_xyz((0, rotation_angle_radians, 0))
    rotated_copy = copy.deepcopy(pcd).rotate(R, center=[0,0,0])
    return rotated_copy


# ==================================================================================================
#  Generate Rotated View
# ==================================================================================================
def cut_pcd(pcd):
    """
    decription:
        generates a virual camera infront of the point cloud and removes all points which
        are not visible from the camera perspective
    input:
        pcd: open3d point cloud
    output:
        cutted POV version point cloud
    """
    if pcd is None:
        print("Error: Point cloud object is None.")
        return None
    
    if pcd.is_empty():
        print("Error: Empty point cloud object detected.")
        return None

    bounds_difference =  np.asarray(pcd.get_max_bound()) - np.asarray(pcd.get_min_bound())
    diameter = np.linalg.norm(bounds_difference)
    c = pcd.get_center()
    camera = o3d.core.Tensor([c[0], c[1], diameter], o3d.core.float64)
    radius = diameter * 100
    legacy_pcd = o3d.t.geometry.PointCloud.from_legacy(pcd)
    _, pt_map = legacy_pcd.hidden_point_removal(camera, radius)
    legacy_pcd = legacy_pcd.select_by_index(pt_map)
    pcd_cut = legacy_pcd.to_legacy()
    return pcd_cut

### 2.2 Example usage <a name="2.2"></a>

In [5]:
# Generate views
example_pcd_file = "example_data/0000.ply"
example_pcd = load_pcd(example_pcd_file)
pcd_rot_90_full = rotate_pcd(example_pcd, 90)
pcd_rot_90_cut = cut_pcd(pcd_rot_90_full)
pcd_rot_270_full = rotate_pcd(example_pcd, 270)
pcd_rot_270_cut = cut_pcd(pcd_rot_270_full)

# Visualize results
draw_pcd(pcd_rot_90_full)
draw_pcd(pcd_rot_90_cut)
draw_pcd(pcd_rot_270_full)
draw_pcd(pcd_rot_270_cut)

## 3.0 Point Cloud to Voxel Grid <a name="3.0"></a>

In this section we show how to transform a point cloud to a voxel grid.

### 3.1 Functions <a name="3.1"></a>

In [6]:
# ==================================================================================================
#  Normalize
# ==================================================================================================
def normalize(pcd, resolution=(64, 128, 64)):
    """
    desrciption:
        normalizes point cloud into the center of the resolution
    input:
        pcd:        open3d point cloud object
        resolution: the resolution of the bounding box in which the point cloud should be normalized
    output:
        normalized open3d point cloud
    """
    if pcd is None:
        print("Error: Point cloud object is None.")
        return None
    
    if pcd.is_empty():
        print("Error: Empty point cloud object detected.")
        return None
    
    bounding_box = pcd.get_axis_aligned_bounding_box()
    center = bounding_box.get_center()
    pcd.translate(-center)
    min_bound = pcd.get_min_bound()
    max_bound = pcd.get_max_bound()
    extents = max_bound - min_bound
    scale_factors = []
    
    for d in extents:
        scale_factors.append(2.0 / d)

    center_bb = tuple(x / 2 for x in resolution)
    center_shifted = tuple(x - 0.5 for x in center_bb)
    scale_factors = np.array(scale_factors)
    scale_full = scale_factors * center_shifted
    points = np.asarray(pcd.points)
    points = points * scale_full
    points = points + center_bb
    pcd.points = o3d.utility.Vector3dVector(np.around(points, decimals=4))
    return pcd


# ==================================================================================================
#  Normalize
# ==================================================================================================
def draw_pcd_in_bounding_box(pcd, min_bound=(0, 0, 0), max_bound=(64, 128, 64)):
    """
    description:
        draws a point cloud with a bounding box
    input:
        pcd:        open3d point cloud object
        min_bound:  min bound of the bounding box
        max_bound:  max bound of the bounding box
    """
    if pcd is None:
        print("Error: Point cloud object is None.")
        return
    
    if pcd.is_empty():
        print("Error: Empty point cloud object detected.")
        return
    
    aabb = o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)
    aabb.color = (1, 0, 0)
    o3d.visualization.draw_geometries([pcd, aabb],
                                      window_name="Bounding Box Viewer",
                                      point_show_normal=False)


# ==================================================================================================
#  Point Cloud to Voxel Grid
# ==================================================================================================
def pcd_to_voxel_grid(pcd, cell_size = 1, lower_bound = (0,0,0), higher_bound=(64,128,64)):
    """
    desctiption:
        creates voxel grid from point cloud
        this function creates the grid from given bound of a bounding box

    input: 
        pcd:            open3d point cloud object
        cell_size:      size of a single cell in the voxel grid
        lower_bound:    3d coordinate of the lower bound of the bounding box
        higher_bound:   3d coordinate of the higher bound of the bounding box

    output:
        generated voxel grid from point cloud
    """
    if pcd is None:
        print("Error: Point cloud object is None.")
        return None
    
    if pcd.is_empty():
        print("Error: Empty point cloud object detected.")
        return None
    
    return o3d.geometry.VoxelGrid.create_from_point_cloud_within_bounds(pcd,
                                                                        voxel_size=cell_size,
                                                                        min_bound=lower_bound,
                                                                        max_bound=higher_bound)


# ==================================================================================================
#  Save Voxel Grid
# ==================================================================================================
def save_voxel_grid(voxel_grid, file_path):
    """
    description:
        saves voxel grid in given file path
        it should be noted, that voxel grids are saved in a ".ply" file
        the resulting file will include all available voxels within the grid

    input:
        voxel_grid: voxels to be saved
        file_path: file path where the voxels will be stored
    """
    if voxel_grid is None:
        print("Error: Point cloud object is None.")
        return
    
    if voxel_grid.is_empty():
        print("Error: Empty point cloud object detected.")
        return
    
    
    o3d.io.write_voxel_grid(file_path, voxel_grid)

### 3.2 Example usage <a name="3.2"></a>

In [7]:
# File paths
example_pcd_file = "example_data/0000.ply"
example_voxel_file = "example_data/voxel_grid.ply"

# Example usage
example_pcd = load_pcd(example_pcd_file)
normalized_pcd = normalize(copy.deepcopy(example_pcd))
draw_pcd_in_bounding_box(example_pcd) # notice the point cloud in the corner around (0,0,0)
draw_pcd_in_bounding_box(normalized_pcd)
example_voxel_grid = pcd_to_voxel_grid(normalized_pcd)
save_voxel_grid(example_voxel_grid, example_voxel_file)

# Notice: Voxel grid are saved as ".ply" files and can be then used like discrete point clouds
loaded_voxels = load_pcd(example_voxel_file)
draw_pcd_in_bounding_box(loaded_voxels)

## 4.0 Voxel to Truncated Disatnce Field <a name="4.0"></a>

This section we show how to transform a voxel grid to a unsigned truncated distance field.

We shortly tried to work with TDFs, but we dropped the approach for various reasons, mentioned in the documentation

### 4.1 Functions <a name="4.1"></a>

In [8]:
# ==================================================================================================
#  Compute unsigned Truncated Distance Field
# ==================================================================================================
def compute_tdf(voxel_pcd, grid_resolution=(64, 128, 64), truncation_distance=5):
    """
    description:
        creates an truncated unsigned distance field from voxel list
        the field is created by calculating the distance of 
        the input point cloud is assumed to be from a voxel grid
    input: 
        voxel_pcd:              open3d point cloud from voxel grid structure
        grid_resolution:        resolution of voxel grid volume
        lower_bound:            3d coordinate of the lower bound of the bounding box
        truncation_distance:    max distance of the TDF, distance above will be truncated
    output:
        truncated unsigned distance field as an numpy volume with size of grid_resolution
    """
    kdtree = cKDTree(voxel_pcd.points)
    x = np.arange(grid_resolution[0])
    y = np.arange(grid_resolution[1])
    z = np.arange(grid_resolution[2])
    grid_x, grid_y, grid_z = np.meshgrid(x, y, z, indexing='ij')
    voxel_centers = np.vstack([grid_x.ravel(), grid_y.ravel(), grid_z.ravel()]).T

    distances, _ = kdtree.query(voxel_centers)
    distances = np.minimum(distances, truncation_distance)

    # Note: order maybe has to be changed and transposed and filpped
    tdf = distances.reshape((grid_resolution))
    return tdf


# ==================================================================================================
#  Save Truncated Distance Field
# ==================================================================================================
def save_tdf(tdf, file_path):
    """
    desctiption:
        saves a TDF at defined file_path
    input: 
        tdf:        TDF stored as a numpy volume
        file_path:  file path to TDF file
    """
    return np.save(file_path, tdf)


# ==================================================================================================
#  Load Truncated Distance Field
# ==================================================================================================
def load_tdf(file_path):
    """
    desctiption:
        loads a TDF
    input: 
        file_path: file path to TDF file
    output:
        loaded TDF as numpy volume
    """
    if not os.path.exists(file_path):
        print(f"Error: The file '{file_path}' does not exist.")
        return None
    
    return np.load(file_path)


# ==================================================================================================
#  Truncated Distance Field to Point Cloud
# ==================================================================================================
def tdf_to_pcd(tdf):
    """
    description:
        transforms TDF to Point Cloud
    input: 
        tdf: truncated distance field stored as a numpy volume
    output:
        reconstructed open3d point cloud
    """
    if tdf is None:
        print("Error: Truncated distance field is None.")
        return None

    zero_indices = np.argwhere(tdf == 0)
    coordinates = zero_indices.astype(np.float32)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(coordinates)
    return pcd

### 4.2 Example usage <a name="4.2"></a>

In [9]:
# File path
example_voxel_file = "example_data/voxel_grid.ply"
example_tdf_file = "example_data/tdf.npy"

# Remember that voxel grids are saved as ".ply" file
voxel_pcd = load_pcd(example_voxel_file)
draw_pcd_in_bounding_box(voxel_pcd)
example_tdf = compute_tdf(voxel_pcd)
save_tdf(example_tdf, example_tdf_file)
loaded_tdf = load_tdf(example_tdf_file)
tdf_pcd = tdf_to_pcd(loaded_tdf)
draw_pcd_in_bounding_box(tdf_pcd)